In [64]:
import cv2
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import random
import skimage
from scipy.stats import norm
from collections import Counter

In [65]:
image1=cv2.imread('Desktop/template/cropped/1.jpg')
image2=cv2.imread('Desktop/template/cropped/2.jpg')
image3=cv2.imread('Desktop/template/cropped/3.jpg')
image4=cv2.imread('Desktop/template/cropped/4.jpg')
image5=cv2.imread('Desktop/template/cropped/5.jpg')
image6=cv2.imread('Desktop/template/cropped/6.jpg')
image7=cv2.imread('Desktop/template/cropped/7.jpg')
image8=cv2.imread('Desktop/template/cropped/8.jpg')
image9=cv2.imread('Desktop/template/cropped/9.jpg')
image10=cv2.imread('Desktop/template/cropped/10.jpg')
image11=cv2.imread('Desktop/template/cropped/11.jpg')
brects1=[[122,126,573,172],[158,174,456,210],[72,215,206,253],[371,215,470,256],[59,255,203,291],[389,259,532,293],[173,298,414,335],[258,342,336,379]]
brects2=[[32,89,564,125],[63,134,552,173],[135,183,547,225],[92,237,219,268],[375,231,521,268],[75,278,171,317],[146,327,370,362]]
brects3=[[35,92,565,132],[37,137,188,174],[393,143,556,180],[36,181,284,218],[317,186,555,221],[34,225,192,264],[215,227,570,266],[30,270,207,309],[215,271,555,312],[87,313,504,355]]
brects4=[[34,86,505,124],[36,133,567,172],[35,182,277,214],[344,189,552,220],[35,231,556,270]]
brects5=[[31,90,554,129],[37,137,553,173],[28,185,562,222],[30,229,524,264],[32,282,456,313],[86,328,492,362]]
brects6=[[237,111,458,150],[126,151,339,187],[236,199,283,229],[505,203,554,238],[234,237,278,272],[505,247,552,281],[228,280,324,311],[393,285,492,317],[235,317,320,348],[424,322,499,355],[211,353,296,387],[421,360,468,392]]
brects7=[[23,79,419,123],[25,132,107,171],[298,131,402,176],[25,183,334,220],[25,229,147,270],[229,231,430,269],[25,279,125,316],[199,275,505,317]]
brects8=[[20,91,436,140],[27,151,108,188],[275,149,543,192],[27,198,330,232],[393,197,470,232],[33,245,162,280],[225,244,539,280],[39,289,166,323],[183,288,474,320]]
brects9=[[222,81,513,123],[166,128,370,173],[168,184,537,223],[56,224,108,258],[393,226,522,269],[53,275,169,317],[418,277,515,316]]
brects10=[[19,80,436,124],[21,131,100,167],[304,135,403,176],[21,181,335,219],[21,230,142,263],[230,231,430,269],[515,231,565,268],[16,275,120,311],[207,280,381,315],[419,279,495,314]]
brects11=[[9,76,422,118],[7,125,99,162],[291,128,398,168],[7,179,337,211],[4,224,132,260],[213,225,434,262],[514,227,564,265],[5,271,107,308],[191,274,494,313]]

In [94]:
def draw_boxes(img, boxes):
    for box in boxes:
        x1, y1, x2, y2 = box
        cv2.rectangle(img, (x1, y1), (x2, y2), (0,0,0), 3)
        #cv2.imwrite('gray/'+'bb.jpg',img)
    plt.imshow(img)

def fit_cropped_areas(cropimg,ind,gray):    
    images = [Image.open(path) for path in [cropimg[i] for i in ind]]
    widths, heights = zip(*(i.size for i in images))
    total_width = sum(widths)
    max_height = max(heights)  
    if gray:
        new_im = Image.new('L', (total_width, max_height))
        x_offset = 0
        for im in images:
            new_im.paste(im, (x_offset,0))
            x_offset += im.size[0]
        resized = cv2.resize(np.array(new_im), dim, interpolation = cv2.INTER_AREA)
        return resized   
    
    new_im = Image.new('RGB', (total_width, max_height))
    x_offset = 0
    for im in images:
        new_im.paste(im, (x_offset,0))
        x_offset += im.size[0]
    resized = cv2.resize(np.array(new_im), dim, interpolation = cv2.INTER_AREA)  
    #resized=plotnoise(resized)
    return resized

def fit_strippedimage(img, ind, gray):
    
    if gray:
        resized = cv2.resize(np.array(Image.open(img[ind]).convert('L')), dim, interpolation = cv2.INTER_AREA)
    else :
        resized = cv2.resize(np.array(Image.open(img[ind])), dim, interpolation = cv2.INTER_AREA)    
        resized = plotnoise(resized)
    return resized

def plotnoise(img):
    noiselist=['gaussian','salt','pepper','s&p','speckle','']
    mode=random.choice(noiselist)
    if mode is not '':
        gimg = skimage.util.random_noise(img, mode=mode)
        #print(gimg)
        return skimage.util.img_as_ubyte(gimg)
    return img

class Template():  
    
    def __init__(self,number,colorimage,bbrect):
        self.number = number
        self.colorimage = colorimage
        self.bbrect = bbrect       
        
    def getbbrect(a):
        return a.bbrect
    
    def getcolorimage(b):
        return b.colorimage
    
    def getnumber(c):
        return c.number
def generate_spot_light_mask(mask_size,
                             position=None,
                             max_brightness=255,
                             min_brightness=0,
                             mode="gaussian",
                             linear_decay_rate=None,
                             speedup=False):
    """
    Generate decayed light mask generated by spot light given position, direction. Multiple spotlights are accepted.
    Args:
        mask_size: tuple of integers (w, h) defining generated mask size
        position: list of tuple of integers (x, y) defining the center of spotlight light position,
                  which is the reference point during rotating
        max_brightness: integer that max brightness in the mask
        min_brightness: integer that min brightness in the mask
        mode: the way that brightness decay from max to min: linear or gaussian
        linear_decay_rate: only valid in linear_static mode. Suggested value is within [0.2, 2]
        speedup: use `shrinkage then expansion` strategy to speed up vale calculation
    Return:
        light_mask: ndarray in float type consisting value from max_brightness to min_brightness. If in 'linear' mode
                    minimum value could be smaller than given min_brightness.
    """
    if position is None:
        position = [(random.randint(0, mask_size[0]), random.randint(0, mask_size[1]))]
    if linear_decay_rate is None:
        if mode == "linear_static":
            linear_decay_rate = random.uniform(0.25, 1)
    assert mode in ["linear", "gaussian"], \
        "mode must be linear_dynamic, linear_static or gaussian"
    mask = np.zeros(shape=(mask_size[1], mask_size[0]), dtype=np.float32)
    if mode == "gaussian":
        mu = np.sqrt(mask.shape[0]**2+mask.shape[1]**2)
        dev = mu / 3.5
        mask = _decay_value_radically_norm_in_matrix(mask_size, position, max_brightness, min_brightness, dev)
    mask = np.asarray(mask, dtype=np.uint8)
    # add median blur
    mask = cv2.medianBlur(mask, 5)
    mask = 255 - mask
    # cv2.imshow("mask", mask)
    # cv2.waitKey(0)
    return mask

def _decay_value_radically_norm_in_matrix(mask_size, centers, max_value, min_value, dev):
    """
    _decay_value_radically_norm function in matrix format
    """
    center_prob = norm.pdf(0, 0, dev)
    x_value_rate = np.zeros((mask_size[1], mask_size[0]))
    for center in centers:
        coord_x = np.arange(mask_size[0])
        coord_y = np.arange(mask_size[1])
        xv, yv = np.meshgrid(coord_x, coord_y)
        dist_x = xv - center[0]
        dist_y = yv - center[1]
        dist = np.sqrt(np.power(dist_x, 2) + np.power(dist_y, 2))
        x_value_rate += norm.pdf(dist, 0, dev) / center_prob
    mask = x_value_rate * (max_value - min_value) + min_value
    mask[mask > 255] = 255
    return mask

def _decay_value_radically_norm(x, centers, max_value, min_value, dev):
    """
    Calculate point value decayed from center following Gaussian decay. If multiple centers are given, value
    from each center sums up while limiting the accumulated value into [0, 255]
    NOTE: assuming light at each center is identical: same brightness and same decay rate
    """
    center_prob = norm.pdf(0, 0, dev)
    x_value_rate = 0
    for center in centers:
        distance = np.sqrt((center[0]-x[0])**2 + (center[1]-x[1])**2)
        x_value_rate += norm.pdf(distance, 0, dev) / center_prob
    x_value = x_value_rate * (max_value - min_value) + min_value
    x_value = 255 if x_value > 255 else x_value
    return x_value

def add_spot_light(image, light_position=None, max_brightness=255, min_brightness=0,
                   mode='gaussian', linear_decay_rate=None, transparency=None):
    """
    Add mask generated from spot light to given image
    """
    if transparency is None:
        transparency = random.uniform(0.5, 0.85)
    #frame = cv2.imread(image)
    frame=image
    height, width, _ = frame.shape
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = generate_spot_light_mask(mask_size=(width, height),
                                    position=light_position,
                                    max_brightness=max_brightness,
                                    min_brightness=min_brightness,
                                    mode=mode,
                                    linear_decay_rate=linear_decay_rate)
    hsv[:, :, 2] = hsv[:, :, 2] * transparency + mask * (1 - transparency)
    frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    frame[frame > 255] = 255
    frame = np.asarray(frame, dtype=np.uint8)
    return frame

In [79]:
templatelist=[Template(1,image1,brects1),Template(2,image2,brects2),Template(3,image3,brects3),Template(4,image4,brects4),Template(5,image5,brects5),Template(6,image6,brects6),Template(7,image7,brects7),Template(8,image8,brects8),Template(9,image9,brects9),Template(10,image10,brects10),Template(11,image11,brects11)]

In [80]:
allcolorcropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/ALL/cluster/'):
        for filename in files:
            allcolorcropimg.append(os.path.join(folder, filename))
allgraycropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/ALL/cluster_gray/'):
        for filename in files:
            allgraycropimg.append(os.path.join(folder, filename))
moregraycropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/ALL/more_gray/resize/'):
        for filename in files:
            moregraycropimg.append(os.path.join(folder, filename))
morecolorcropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/ALL/more/resize/'):
        for filename in files:
            morecolorcropimg.append(os.path.join(folder, filename))
smallgraycropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/ALL/smallcluster_gray/'):
        for filename in files:
            smallgraycropimg.append(os.path.join(folder, filename))
smallcolorcropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/ALL/smallcluster/'):
        for filename in files:
            smallcolorcropimg.append(os.path.join(folder, filename))
# more=[]
# for folder, subs, files in os.walk('Desktop/threshold/more/'):
#         for filename in files:
#             more.append(os.path.join(folder, filename)) 
# alls=[]
# for folder, subs, files in os.walk('Desktop/threshold/all/'):
#         for filename in files:
#             alls.append(os.path.join(folder, filename)) 
# smallcluster=[]
# for folder, subs, files in os.walk('Desktop/threshold/smallcluster/'):
#         for filename in files:
#             smallcluster.append(os.path.join(folder, filename)) 
more=[]
for folder, subs, files in os.walk('Desktop/diff/more/'):
        for filename in files:
            more.append(os.path.join(folder, filename)) 
alls=[]
for folder, subs, files in os.walk('Desktop/diff/all/'):
        for filename in files:
            alls.append(os.path.join(folder, filename)) 
smallcluster=[]
for folder, subs, files in os.walk('Desktop/diff/smallcluster/'):
        for filename in files:
            smallcluster.append(os.path.join(folder, filename)) 

In [87]:
for count in range(100):
    t=random.choice(templatelist)
    img_data=t.getcolorimage()
    im=img_data.copy()
    num=t.getnumber()
    gray=cv2.imread('Desktop/diff/temp/'+str(num)+'.jpg', 0)
    #gray=cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    #noimg=plotnoise(im)
    noimg=im.copy()
    for i,(startX, startY, endX, endY) in enumerate(t.getbbrect()):
        #noimg[startY:endY, startX:endX] = [255,255,255]
        #gray[startY:endY, startX:endX] = [255]
        height = endY - startY
        width = endX - startX
        dim = (width, height)
        
        if width < 100 :
            
            ind=random.sample(range(1,len(morecolorcropimg)), int(dim[0]/random.choice(range(18,25))))
            noimg[startY:endY, startX:endX] = fit_cropped_areas(morecolorcropimg,ind,False)
            gray[startY:endY, startX:endX] = fit_cropped_areas(more,ind, True)   
        
        elif width > 100 and width <150 :
            
            indexsyn=random.choice(range(1,len(smallcolorcropimg)))
            noimg[startY:endY, startX:endX] = fit_strippedimage(smallcolorcropimg,indexsyn,False)
            gray[startY:endY, startX:endX] = fit_strippedimage(smallcluster,indexsyn, True) 
        
        else :
            
            indexsyn=random.choice(range(1,len(allcolorcropimg)))
            noimg[startY:endY, startX:endX] = fit_strippedimage(allcolorcropimg,indexsyn,False)
            gray[startY:endY, startX:endX] = fit_strippedimage(alls,indexsyn, True)
            
            
    #color=add_spot_light(noimg)   
    cv2.imwrite('Desktop/Thesis/Motorplates_Images/ksbgansyndata/A/img'+str(count+401)+'.jpg', noimg)
    cv2.imwrite('Desktop/Thesis/Motorplates_Images/ksbgansyndata/B/img'+str(count+401)+'.jpg', gray)
    
    #cv2.imwrite('ksbsynimages/projected/img'+str(count+1)+'.jpg', im)
    #cv2.imwrite('ksbsynimages/caps/img'+str(count+1)+'.jpg', im)
    #cv2.imwrite('ksbsynimages/All/img'+str(count+1)+'.jpg', im)

In [95]:
for count in range(100):
    t=random.choice(templatelist)
    img_data=t.getcolorimage()
    im=img_data.copy()
    num=t.getnumber()
    gray=cv2.imread('Desktop/diff/temp/'+str(num)+'.jpg', 0)
    #gray=cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    #noimg=plotnoise(im)
    noimg=im.copy()
    for i,(startX, startY, endX, endY) in enumerate(t.getbbrect()):
        #noimg[startY:endY, startX:endX] = [255,255,255]
        #gray[startY:endY, startX:endX] = [255]
        height = endY - startY
        width = endX - startX
        dim = (width, height)
        ind=random.sample(range(1,len(morecolorcropimg)), int(dim[0]/random.choice(range(18,25))))
        noimg[startY:endY, startX:endX] = fit_cropped_areas(morecolorcropimg,ind,False)
        gray[startY:endY, startX:endX] = fit_cropped_areas(more,ind, True)            
            
    #color=add_spot_light(noimg)   
    cv2.imwrite('Desktop/Thesis/Motorplates_Images/ksbgansyndata/A/img'+str(count+501)+'.jpg', noimg)
    cv2.imwrite('Desktop/Thesis/Motorplates_Images/ksbgansyndata/B/img'+str(count+501)+'.jpg', gray)
    
    #cv2.imwrite('ksbsynimages/projected/img'+str(count+1)+'.jpg', im)
    #cv2.imwrite('ksbsynimages/caps/img'+str(count+1)+'.jpg', im)
    #cv2.imwrite('ksbsynimages/All/img'+str(count+1)+'.jpg', im)

In [ ]:
imprintedcropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/char_imprinted/resize/'):
        for filename in files:
            imprintedcropimg.append(os.path.join(folder, filename))
imprintedclearcropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/char_imprinted/opt/'):
        for filename in files:
            imprintedclearcropimg.append(os.path.join(folder, filename))
projectedcropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/char_projected/resize/'):
        for filename in files:
            projectedcropimg.append(os.path.join(folder, filename))
projectedclearcropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/char_projected/opt/'):
        for filename in files:
            projectedclearcropimg.append(os.path.join(folder, filename))
CAPScropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/caps/resize/'):
        for filename in files:
            CAPScropimg.append(os.path.join(folder, filename))
CAPSclearcropimg=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/caps/opt/'):
        for filename in files:
            CAPSclearcropimg.append(os.path.join(folder, filename))
alllist=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/ALL/resize/'):
        for filename in files:
            alllist.append(os.path.join(folder, filename))
allclearlist=[]
for folder, subs, files in os.walk('Downloads/ksb-cropped/train/characters/ALL/B&W/resize'):
        for filename in files:
            allclearlist.append(os.path.join(folder, filename))

In [ ]:
templist=[]
for folder, subs, files in os.walk('downloads/ksb-cropped/validation/Readable/testsamples/resize/'):
        for filename in files:
            templist.append(os.path.join(folder, filename))

In [ ]:
# seqs = ['old_people', 'young_people', 'mid_age']
# random.choice(random.choices(seqs, weights=map(len, seqs)))
# Counter(random.choice(random.choices(seqs, weights=map(len, seqs))) for _ in range(10**3))

draw = choice(range(1,len(smallcolorcropimg)), 1)

In [ ]:
gray = cv2.cvtColor(cv2.imread('downloads/ksb-cropped/train/characters/ALL/cluster/28.jpg'), cv2.COLOR_RGB2GRAY)
cv2.imwrite('downloads/ksb-cropped/train/characters/ALL/cluster_gray/28.jpg', gray)

In [ ]:
for img in templist:
    outfile = 'downloads/ksb-cropped/validation/Readable/testsamples/gray/'+ os.path.splitext(img)[0].split('/')[-1] + ".jpg"
    roi=cv2.imread(img)
    gray = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
        #Applying adaptive threshold
#     filtered = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 15, 3)
#     kernel = np.ones((1, 1), np.uint8)
#         #Smoothening and noise removal
#     opening = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
#     closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)         
#         #Applying different thresholding
#         #ret1, th1 = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
#     th1 = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,27,5)
#     ret2, th2 = cv2.threshold(th1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#     blur = cv2.GaussianBlur(th2, (1, 1), 0)
#     ret3, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)        
#         #Combining both thresholding
#     or_image = cv2.bitwise_or(th3, closing)
#     inv_img = (255 - or_image)
    cv2.imwrite(outfile, gray)    

In [ ]:
templatelist=[t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11]

In [ ]:
t1=Template(1,image1,brects1)
t2=Template(2,image2,brects2)
t3=Template(3,image3,brects3)
t4=Template(4,image4,brects4)
t5=Template(5,image5,brects5)
t6=Template(6,image6,brects6)
t7=Template(7,image7,brects7)
t8=Template(8,image8,brects8)
t9=Template(9,image9,brects9)
t10=Template(10,image10,brects10)
t11=Template(11,image11,brects11)

In [ ]:
rects=[[107,110,519,160],[140,163,417,200],[64,202,190,240],[336,206,425,242],[64,245,190,279],[352,245,478,282],[136,285,420,322],[202,325,342,364]]

In [ ]:
draw_boxes(im11,rects11)